In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import mbo_utilities as mbo
import tifffile

Image(value=b'version https://git-lfs.github.com/spec/...', height='55', width='300')

Valid,Device,Type,Backend,Driver
✅ (default),NVIDIA RTX A4000,DiscreteGPU,Vulkan,560.94
✅,NVIDIA RTX A4000,DiscreteGPU,D3D12,
✅,NVIDIA RTX A4000,DiscreteGPU,D3D12,
❗ limited,Microsoft Basic Render Driver,CPU,D3D12,
❌,NVIDIA RTX A4000/PCIe/SSE2,Unknown,OpenGL,4.6.0 NVIDIA 560.94


In [2]:
# original implementation in v1.0.0
v1_files = mbo.get_files(r"D:\W2_DATA\kbarber\2025_03_01\mk301\assembled", "07")
v1_data = tifffile.memmap(v1_files[0])
v1_data.shape

(64594, 448, 448)

In [3]:
# original implementation in v1.0.0
v2_files = mbo.get_files(r"D:\demo\masknmf\assembled\roi_1", "07")
v2_data = tifffile.memmap(v2_files[0])
v2_data.shape

(2135, 448, 224)

In [2]:
scan = mbo.read_scan(r"D:\W2_DATA\kbarber\2025_03_01\mk301\green",)

Initializing MBO Scan with parameters:
roi: None, fix_phase: True, phasecorr_method: frame, border: 3, upsample: 1, max_offset: 4


Raw tiff fully read.


In [5]:
%%timeit
scan[:200, 0, :, :]

2.57 s ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
import timeit

index_cases = {
    "Single frame (0,0,:,:)"             : "scan[0, 0, :, :]",
    "Batch 5 frames, same Z (20:25,0,:,:)": "scan[20:25, 0, :, :]",
     "All Z for frame 20 (20,:,:,:)"      : "scan[20, :, :, :]",
     "Single Z stack (slice) (20:25,:,:,:)": "scan[20:25, :, :, :]",
     "200 frame Z stack (slice) (20:25,:,:,:)": "scan[20:25, :, :, :]",
}

timing_results = {}

for label, stmt in index_cases.items():
    t = timeit.timeit(stmt, globals=globals(), number=10)
    avg_time = t / 10
    fps = 1 / avg_time if avg_time > 0 else float('inf')
    timing_results[label] = {"Avg Time (s)": avg_time, "Est. FPS": fps}

df = pd.DataFrame.from_dict(timing_results, orient="index")

In [9]:
df

[autoreload of mbo_utilities.graphics.imgui failed: Traceback (most recent call last):
  File "C:\Users\RBO\repos\mbo_utilities\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 280, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\RBO\repos\mbo_utilities\.venv\Lib\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 533, in maybe_reload_module
    new_source_code = f.read()
                      ^^^^^^^^
  File "C:\Users\RBO\AppData\Roaming\uv\python\cpython-3.11.12-windows-x86_64-none\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 34105: character maps to <undefined>
]


,Avg Time (s),Est. FPS
"Single frame (0,0,:,:)",0.012702,78.725032
"Batch 5 frames, same Z (20:25,0,:,:)",0.065995,15.152654
"All Z for frame 20 (20,:,:,:)",0.176351,5.670518
"Single Z stack (slice) (20:25,:,:,:)",0.900657,1.110301
